In [1]:
!pip install transformers sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 1.2/1.2 MB 9.5 MB/s eta 0:00:00
     ------------------------------------- 977.5/977.5 kB 10.3 MB/s eta 0:00:00
     -------------------------------------- 172.3/172.3 MB 7.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125960 sha256=2f3103f25889dd8b441695e7c08248cbdf579855f2005443d6ca3a213e76d657
  Stored in directory: c:\users\prashanthkumarreddy\appdata\local\pip\cache\wheels\0a\f5\dd\9d00836c4e9e279c2a59d5b0ab72dafa66cbc626a327c550dd
Successfully built sentence-transformers


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_essay_data(verbose=False, words_count=500):
    PATH = "drive/MyDrive/Fall_2023_Project/ghostbuster/"
    PATH = ""
    ESSAY_DATA_PATH = PATH + "essay/data/"
    required_data = ["gpt", "human", "claude"]
    SKIP = ["logprobs"]
    docs, names, labels = [], [], []

    for split in required_data:
        for filename in os.listdir(ESSAY_DATA_PATH + split):
            filepath = os.path.join(ESSAY_DATA_PATH, split, filename)

            if filename in SKIP:
                continue
                
            with open(filepath, encoding="utf8") as f:
                doc = f.read()

            doc = doc.split()[:words_count]
            doc = " ".join(doc)

            filepath = filepath.replace(PATH, "")

            label = 0
            if split == "human":
                label = 1

            if verbose:
                print(filepath)
                
            names.append(filepath)
            docs.append(doc)
            labels.append(label)

    return {
      "names": names,
      "docs": docs,
      "labels": labels
  }


In [3]:
import pandas as pd

res = get_essay_data()

df = pd.DataFrame({
    "txt": res["docs"],
    "filename": res["names"],
    "label": res["labels"]
})
df.shape

(4332, 3)

In [4]:
df.head()

,txt,filename,label
0,Introduction Having children is a crucial aspe...,essay/data/gpt\0.txt,0
1,"During the Victorian era in England, women wer...",essay/data/gpt\1.txt,0
2,Enzymes are essential biological catalysts tha...,essay/data/gpt\10.txt,0
3,Memorials represent a crucial part of cultural...,essay/data/gpt\100.txt,0
4,English language has three functional categori...,essay/data/gpt\1000.txt,0


In [5]:
df.label.value_counts()

0    2888
1    1444
Name: label, dtype: int64

In [6]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
batch_1 = df[4000:]

In [8]:
#df['txt'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [9]:
tokenized = batch_1['txt'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512)))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [10]:
tokenized.shape, len(tokenized)

((332,), 332)

In [11]:
max_len = max([len(i) for i in tokenized.values])
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [12]:
np.array(padded).shape

(332, 512)

In [13]:
padded

array([[ 101, 3235, 9678, ..., 2116, 8771,  102],
       [ 101, 1996, 2149, ..., 2162, 2006,  102],
       [ 101, 2129, 2515, ...,    0,    0,    0],
       ...,
       [ 101, 2653, 2003, ..., 1998, 2529,  102],
       [ 101, 8106, 1998, ...,    0,    0,    0],
       [ 101, 5418, 8035, ...,    0,    0,    0]])

In [14]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(332, 512)

In [15]:
attention_mask[:5]

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])

In [16]:
from datetime import datetime


start = datetime.now()
input_ids = torch.tensor(padded)
attention_masks = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_masks)
end = datetime.now()
end - start

datetime.timedelta(seconds=57, microseconds=863556)

In [17]:
1590/60

26.5

In [18]:
features = last_hidden_states[0][:,0,:].numpy()

In [19]:
features

array([[-0.17253688,  0.26612163, -0.46258152, ...,  0.14168829,
         0.3659189 ,  0.6282809 ],
       [-0.48860657, -0.17359151, -0.5914039 , ..., -0.14058182,
         0.45984328,  0.2062838 ],
       [-0.41730145, -0.02333027, -0.53475994, ..., -0.23151751,
         0.3446241 ,  0.62262744],
       ...,
       [-0.2232965 ,  0.12151001, -0.59437186, ..., -0.06084322,
         0.31083953,  0.6505159 ],
       [-0.49223104, -0.06833295, -0.5358462 , ..., -0.37378776,
         0.1767093 ,  0.31469285],
       [-0.46127778,  0.10093249, -0.37873298, ..., -0.22947782,
         0.19412641,  0.53395593]], dtype=float32)

In [20]:
len(features)

332

In [21]:
len(batch_1.label)

332

In [22]:
tmp_df = pd.DataFrame({
    'sentence_embeddings': features.tolist(),
    'label': batch_1.label
})

In [25]:
(tmp_df.sentence_embeddings.iloc[0] == features[0]).sum()

768

In [26]:
tmp_df.head(5)

,sentence_embeddings,label
4000,"[-0.17253687977790833, 0.26612162590026855, -0...",0
4001,"[-0.4886065721511841, -0.1735915094614029, -0....",0
4002,"[-0.4173014461994171, -0.023330267518758774, -...",0
4003,"[-0.3777746260166168, -0.020216505974531174, -...",0
4004,"[-0.2621786296367645, -0.06937450170516968, -0...",0


In [29]:
import dill as pickle

pickle.dump(tmp_df, open('features/essay_sentence_embeddings_batch2', 'wb'))

In [30]:
batch_1.label.value_counts()

0    332
Name: label, dtype: int64